In [ ]:
!pip install flask pyngrok
!pip install gTTS


In [ ]:
from flask import Flask, request, render_template, jsonify, send_file, redirect, url_for, session, flash
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
import os
import torch
import torchaudio
import joblib
import librosa
import numpy as np
import pickle
import soundfile as sf
import requests
import io
import sqlite3

from torch import nn
from transformers import Wav2Vec2Model, Wav2Vec2Processor, Wav2Vec2ForCTC
from huggingface_hub import hf_hub_download
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pyngrok import ngrok
from gtts import gTTS  # Import Google Text-to-Speech




# Set your ngrok auth token
ngrok.set_auth_token("2e2WtPpQDF8CKyboBNM8GHkxHPO_ZSzKBHe3LmExvbU86DBY")

# Initialize Flask app
app = Flask(__name__)
app.secret_key = 'your_secret_key'
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['TTS_FOLDER'] = 'tts_outputs'  # Folder for speech synthesis outputs
app.config['ALLOWED_EXTENSIONS'] = {'wav', 'mp3'}
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs(app.config['TTS_FOLDER'], exist_ok=True)  # Create TTS folder

# Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Database setup function
def init_db():
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()

    # Create users table if it doesn't exist
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL
    )
    ''')

    conn.commit()
    conn.close()


# Load models --------------------------------

# 1. Emotion Classification Model
# Repo & file info
REPO_ID = "mahesh006/classification"
MODEL_FILENAME = "emotion_classifier_wav2vec2.pth"
ENCODER_FILENAME = "emotion_label_encoder.pkl"

# Download model and label encoder
MODEL_PATH = hf_hub_download(repo_id=REPO_ID, filename=MODEL_FILENAME)
ENCODER_PATH = hf_hub_download(repo_id=REPO_ID, filename=ENCODER_FILENAME)

# Load processor and encoder
emotion_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
label_encoder = joblib.load(ENCODER_PATH)

# Define emotion model
class EmotionClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.wav2vec2 = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
        self.classifier = nn.Linear(self.wav2vec2.config.hidden_size, num_classes)

    def forward(self, input_values):
        with torch.no_grad():
            hidden_states = self.wav2vec2(input_values).last_hidden_state
        pooled_output = hidden_states.mean(dim=1)
        return self.classifier(pooled_output)

# Initialize and load emotion model
emotion_model = EmotionClassifier(num_classes=len(label_encoder.classes_)).to(DEVICE)
emotion_model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
emotion_model.eval()

# 2. Transcription Models
# Telugu Transcription Model (existing)
telugu_processor = Wav2Vec2Processor.from_pretrained("addy88/wav2vec2-telugu-stt")
telugu_model = Wav2Vec2ForCTC.from_pretrained("addy88/wav2vec2-telugu-stt")

# Hindi Transcription Model (new)
hindi_processor = Wav2Vec2Processor.from_pretrained("addy88/hindi-wav2vec2-stt")
hindi_model = Wav2Vec2ForCTC.from_pretrained("addy88/hindi-wav2vec2-stt")

# English Transcription Model (new)
english_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
english_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# 3. Translation Models
CACHE_DIR = "./hf_model"
os.makedirs(CACHE_DIR, exist_ok=True)

# Download helper function
def download_from_hf(filename, repo):
    url = f"https://huggingface.co/{repo}/resolve/main/{filename}"
    out_path = os.path.join(CACHE_DIR, filename)
    if not os.path.exists(out_path):
        print(f"⬇️  Downloading {filename} from {repo}...")
        r = requests.get(url)
        with open(out_path, "wb") as f:
            f.write(r.content)
    return out_path

# PART A: Telugu to English (existing model)
TE_HF_REPO = "mahesh006/lstm"
te_weights_path = download_from_hf("nmt_lstm_te_to_en.weights.h5", TE_HF_REPO)
te_src_tok_path = download_from_hf("src_tokenizer.pkl", TE_HF_REPO)
te_tgt_tok_path = download_from_hf("tgt_tokenizer.pkl", TE_HF_REPO)

# Load Telugu-English tokenizers
with open(te_src_tok_path, "rb") as f:
    te_src_tok = pickle.load(f)
with open(te_tgt_tok_path, "rb") as f:
    te_tgt_tok = pickle.load(f)

te_num_src_tokens = len(te_src_tok.word_index) + 1
te_num_tgt_tokens = len(te_tgt_tok.word_index) + 1
te_max_src_len = 50
te_max_tgt_len = 50
te_latent_dim = 256

# PART B: Hindi to English (new model) - Setup similar structure
HI_EN_HF_REPO = "mahesh006/hinditoenglish" # Placeholder, use actual repo name
HI_EN_WEIGHTS_FILENAME = "nmt_lstm_hi_to_en_finetuned.weights.h5"
HI_EN_SRC_TOK_FILENAME = "tokenizer_src_hi.pkl"
HI_EN_TGT_TOK_FILENAME = "tokenizer_tgt_en.pkl"

print("\n--- Setting up Hindi-English Model ---")
# Attempt to download the necessary files
hi_en_weights_path = download_from_hf(HI_EN_WEIGHTS_FILENAME, HI_EN_HF_REPO)
hi_en_src_tok_path = download_from_hf(HI_EN_SRC_TOK_FILENAME, HI_EN_HF_REPO)
hi_en_tgt_tok_path = download_from_hf(HI_EN_TGT_TOK_FILENAME, HI_EN_HF_REPO)


# Load Hindi-English tokenizers from local files (simulate downloading)
try:
    # Try to load if they exist
    with open(hi_en_src_tok_path, "rb") as f:
        hi_en_src_tok = pickle.load(f)
    with open(hi_en_tgt_tok_path, "rb") as f:
        hi_en_tgt_tok = pickle.load(f)
except (FileNotFoundError, EOFError):
    # For testing purposes, create dummy tokenizers
    print("Using placeholder Hindi-English tokenizers")
    # In production, this would be replaced with proper downloading
    hi_en_src_tok = te_src_tok  # Using Telugu tokenizer as placeholder
    hi_en_tgt_tok = te_tgt_tok  # Using Telugu tokenizer as placeholder

hi_en_num_src_tokens = len(hi_en_src_tok.word_index) + 1
hi_en_num_tgt_tokens = len(hi_en_tgt_tok.word_index) + 1
hi_en_max_src_len = 50
hi_en_max_tgt_len = 50
hi_en_latent_dim = 256

# PART C: English to Hindi (new model) - Setup similar structure
EN_HI_HF_REPO = "mahesh006/englishtohindi" # Placeholder, use actual repo name
EN_HI_WEIGHTS_FILENAME = "nmt_lstm_en_to_hi.weights.h5"
EN_HI_SRC_TOK_FILENAME = "tokenizer_src_en.pkl"
EN_HI_TGT_TOK_FILENAME = "tokenizer_tgt_hi.pkl"

print("\n--- Setting up English-Hindi Model ---")
# Attempt to download the necessary files
en_hi_weights_path = download_from_hf(EN_HI_WEIGHTS_FILENAME, EN_HI_HF_REPO)
en_hi_src_tok_path = download_from_hf(EN_HI_SRC_TOK_FILENAME, EN_HI_HF_REPO)
en_hi_tgt_tok_path = download_from_hf(EN_HI_TGT_TOK_FILENAME, EN_HI_HF_REPO)

# Load English-Hindi tokenizers from local files (simulate downloading)
try:
    # Try to load if they exist
    with open(en_hi_src_tok_path, "rb") as f:
        en_hi_src_tok = pickle.load(f)
    with open(en_hi_tgt_tok_path, "rb") as f:
        en_hi_tgt_tok = pickle.load(f)
except (FileNotFoundError, EOFError):
    # For testing purposes, create dummy tokenizers
    print("Using placeholder English-Hindi tokenizers")
    # In production, this would be replaced with proper downloading
    en_hi_src_tok = te_tgt_tok  # Using Telugu tokenizer as placeholder
    en_hi_tgt_tok = te_src_tok  # Using Telugu tokenizer as placeholder

en_hi_num_src_tokens = len(en_hi_src_tok.word_index) + 1
en_hi_num_tgt_tokens = len(en_hi_tgt_tok.word_index) + 1
en_hi_max_src_len = 50
en_hi_max_tgt_len = 50
en_hi_latent_dim = 256

# Function to build translation models for any language pair
def build_translation_model(num_src_tokens, num_tgt_tokens, latent_dim):
    # Encoder
    encoder_inputs = Input(shape=(None,), name="encoder_inputs")
    enc_emb = Embedding(num_src_tokens, latent_dim, mask_zero=True, name="enc_embedding")(encoder_inputs)
    encoder_lstm = LSTM(latent_dim, return_state=True, name="encoder_lstm")
    _, state_h, state_c = encoder_lstm(enc_emb)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = Input(shape=(None,), name="decoder_inputs")
    dec_emb = Embedding(num_tgt_tokens, latent_dim, mask_zero=True, name="dec_embedding")(decoder_inputs)
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name="decoder_lstm")
    decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
    decoder_dense = Dense(num_tgt_tokens, activation="softmax", name="decoder_dense")
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model, encoder_inputs, encoder_states, decoder_inputs, decoder_lstm, decoder_dense

# Build Telugu-English translation model (existing)
te_model, te_encoder_inputs, te_encoder_states, te_decoder_inputs, te_decoder_lstm, te_decoder_dense = build_translation_model(
    te_num_src_tokens, te_num_tgt_tokens, te_latent_dim
)
te_model.load_weights(te_weights_path)

# Telugu-English inference models
te_encoder_model = Model(te_encoder_inputs, te_encoder_states)

te_decoder_state_input_h = Input(shape=(te_latent_dim,), name="dec_state_h")
te_decoder_state_input_c = Input(shape=(te_latent_dim,), name="dec_state_c")
te_dec_states_inputs = [te_decoder_state_input_h, te_decoder_state_input_c]

te_dec_emb_inf = te_model.get_layer("dec_embedding")(te_decoder_inputs)
te_dec_outputs_inf, te_h_inf, te_c_inf = te_model.get_layer("decoder_lstm")(
    te_dec_emb_inf, initial_state=te_dec_states_inputs
)
te_dec_outputs_inf = te_model.get_layer("decoder_dense")(te_dec_outputs_inf)

te_decoder_model = Model(
    [te_decoder_inputs] + te_dec_states_inputs,
    [te_dec_outputs_inf, te_h_inf, te_c_inf]
)

# Reverse lookup dictionaries for Telugu-English
te_rev_src_index = {i: w for w, i in te_src_tok.word_index.items()}
te_rev_tgt_index = {i: w for w, i in te_tgt_tok.word_index.items()}
te_tgt_word_index = te_tgt_tok.word_index

# Build Hindi-English translation model (new)
hi_en_model, hi_en_encoder_inputs, hi_en_encoder_states, hi_en_decoder_inputs, hi_en_decoder_lstm, hi_en_decoder_dense = build_translation_model(
    hi_en_num_src_tokens, hi_en_num_tgt_tokens, hi_en_latent_dim
)
# Load weights if available
try:
    hi_en_model.load_weights(hi_en_weights_path)
except:
    print("Hindi-English model weights not found. Using uninitialized model.")

# Hindi-English inference models
hi_en_encoder_model = Model(hi_en_encoder_inputs, hi_en_encoder_states)

hi_en_decoder_state_input_h = Input(shape=(hi_en_latent_dim,), name="hi_en_dec_state_h")
hi_en_decoder_state_input_c = Input(shape=(hi_en_latent_dim,), name="hi_en_dec_state_c")
hi_en_dec_states_inputs = [hi_en_decoder_state_input_h, hi_en_decoder_state_input_c]

hi_en_dec_emb_inf = hi_en_model.get_layer("dec_embedding")(hi_en_decoder_inputs)
hi_en_dec_outputs_inf, hi_en_h_inf, hi_en_c_inf = hi_en_model.get_layer("decoder_lstm")(
    hi_en_dec_emb_inf, initial_state=hi_en_dec_states_inputs
)
hi_en_dec_outputs_inf = hi_en_model.get_layer("decoder_dense")(hi_en_dec_outputs_inf)

hi_en_decoder_model = Model(
    [hi_en_decoder_inputs] + hi_en_dec_states_inputs,
    [hi_en_dec_outputs_inf, hi_en_h_inf, hi_en_c_inf]
)

# Reverse lookup dictionaries for Hindi-English
hi_en_rev_src_index = {i: w for w, i in hi_en_src_tok.word_index.items()}
hi_en_rev_tgt_index = {i: w for w, i in hi_en_tgt_tok.word_index.items()}
hi_en_tgt_word_index = hi_en_tgt_tok.word_index

# Build English-Hindi translation model (new)
en_hi_model, en_hi_encoder_inputs, en_hi_encoder_states, en_hi_decoder_inputs, en_hi_decoder_lstm, en_hi_decoder_dense = build_translation_model(
    en_hi_num_src_tokens, en_hi_num_tgt_tokens, en_hi_latent_dim
)
# Load weights if available
try:
    en_hi_model.load_weights(en_hi_weights_path)
except:
    print("English-Hindi model weights not found. Using uninitialized model.")

# English-Hindi inference models
en_hi_encoder_model = Model(en_hi_encoder_inputs, en_hi_encoder_states)

en_hi_decoder_state_input_h = Input(shape=(en_hi_latent_dim,), name="en_hi_dec_state_h")
en_hi_decoder_state_input_c = Input(shape=(en_hi_latent_dim,), name="en_hi_dec_state_c")
en_hi_dec_states_inputs = [en_hi_decoder_state_input_h, en_hi_decoder_state_input_c]

en_hi_dec_emb_inf = en_hi_model.get_layer("dec_embedding")(en_hi_decoder_inputs)
en_hi_dec_outputs_inf, en_hi_h_inf, en_hi_c_inf = en_hi_model.get_layer("decoder_lstm")(
    en_hi_dec_emb_inf, initial_state=en_hi_dec_states_inputs
)
en_hi_dec_outputs_inf = en_hi_model.get_layer("decoder_dense")(en_hi_dec_outputs_inf)

en_hi_decoder_model = Model(
    [en_hi_decoder_inputs] + en_hi_dec_states_inputs,
    [en_hi_dec_outputs_inf, en_hi_h_inf, en_hi_c_inf]
)

# Reverse lookup dictionaries for English-Hindi
en_hi_rev_src_index = {i: w for w, i in en_hi_src_tok.word_index.items()}
en_hi_rev_tgt_index = {i: w for w, i in en_hi_tgt_tok.word_index.items()}
en_hi_tgt_word_index = en_hi_tgt_tok.word_index

# Processing functions --------------------------------

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

def predict_emotion(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

    inputs = emotion_processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt").input_values.to(DEVICE)

    with torch.no_grad():
        logits = emotion_model(inputs)
        pred_id = torch.argmax(logits, dim=1).item()
        return label_encoder.inverse_transform([pred_id])[0]

# Transcription functions for different languages
def transcribe_audio_telugu(audio_path):
    audio_input, _ = librosa.load(audio_path, sr=16000)
    input_values = telugu_processor(audio_input, sampling_rate=16000, return_tensors="pt").input_values
    with torch.no_grad():
        logits = telugu_model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = telugu_processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription

def transcribe_audio_hindi(audio_path):
    audio_input, _ = librosa.load(audio_path, sr=16000)
    input_values = hindi_processor(audio_input, sampling_rate=16000, return_tensors="pt").input_values
    with torch.no_grad():
        logits = hindi_model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = hindi_processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription

def transcribe_audio_english(audio_path):
    audio_input, _ = librosa.load(audio_path, sr=16000)
    input_values = english_processor(audio_input, sampling_rate=16000, return_tensors="pt").input_values
    with torch.no_grad():
        logits = english_model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = english_processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription

# Translation functions
def translate_telugu_to_english(input_text):
    seq = te_src_tok.texts_to_sequences([input_text])
    seq = pad_sequences(seq, maxlen=te_max_src_len, padding="post")
    states_val = te_encoder_model.predict(seq)

    target_seq = np.array([[te_tgt_word_index["<start>"]]])
    decoded_sentence = []

    for _ in range(te_max_tgt_len):
        output_tokens, h, c = te_decoder_model.predict([target_seq] + states_val)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = te_rev_tgt_index.get(sampled_token_index, "")
        if sampled_word == "<end>" or sampled_word == "":
            break
        decoded_sentence.append(sampled_word)
        target_seq = np.array([[sampled_token_index]])
        states_val = [h, c]

    return " ".join(decoded_sentence)

def translate_hindi_to_english(input_text):
    seq = hi_en_src_tok.texts_to_sequences([input_text])
    seq = pad_sequences(seq, maxlen=hi_en_max_src_len, padding="post")
    states_val = hi_en_encoder_model.predict(seq)

    target_seq = np.array([[hi_en_tgt_word_index["<start>"]]])
    decoded_sentence = []

    for _ in range(hi_en_max_tgt_len):
        output_tokens, h, c = hi_en_decoder_model.predict([target_seq] + states_val)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = hi_en_rev_tgt_index.get(sampled_token_index, "")
        if sampled_word == "<end>" or sampled_word == "":
            break
        decoded_sentence.append(sampled_word)
        target_seq = np.array([[sampled_token_index]])
        states_val = [h, c]

    return " ".join(decoded_sentence)

def translate_english_to_hindi(input_text):
    seq = en_hi_src_tok.texts_to_sequences([input_text])
    seq = pad_sequences(seq, maxlen=en_hi_max_src_len, padding="post")
    states_val = en_hi_encoder_model.predict(seq)

    target_seq = np.array([[en_hi_tgt_word_index["<start>"]]])
    decoded_sentence = []

    for _ in range(en_hi_max_tgt_len):
        output_tokens, h, c = en_hi_decoder_model.predict([target_seq] + states_val)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = en_hi_rev_tgt_index.get(sampled_token_index, "")
        if sampled_word == "<end>" or sampled_word == "":
            break
        decoded_sentence.append(sampled_word)
        target_seq = np.array([[sampled_token_index]])
        states_val = [h, c]

    return " ".join(decoded_sentence)

# Text-to-Speech function
def generate_speech(text, lang_code, output_filename):
    """
    Generate speech from text using Google Text-to-Speech

    Args:
        text (str): Text to convert to speech
        lang_code (str): Language code (e.g., 'en', 'hi', 'te')
        output_filename (str): Path to save the audio file

    Returns:
        str: Path to the generated audio file
    """
    if not text:
        return None

    try:
        tts = gTTS(text=text, lang=lang_code, slow=False)
        tts.save(output_filename)
        return output_filename
    except Exception as e:
        print(f"TTS Error: {str(e)}")
        return None

# Language detection function (simple version based on character set)
def detect_language(text):
    # Unicode ranges for different scripts
    devanagari_range = range(0x0900, 0x097F)  # Hindi
    telugu_range = range(0x0C00, 0x0C7F)      # Telugu

    # Count characters in each range
    devanagari_count = sum(1 for char in text if ord(char) in devanagari_range)
    telugu_count = sum(1 for char in text if ord(char) in telugu_range)

    # Determine the dominant script
    if devanagari_count > telugu_count:
        return "hindi"
    elif telugu_count > 0:
        return "telugu"
    else:
        return "english"  # Default to English if no specific script is detected

# Get language code for TTS
def get_tts_lang_code(language):
    # Mapping of our language names to Google TTS language codes
    lang_map = {
        "english": "en",
        "hindi": "hi",
        "telugu": "te"
    }
    return lang_map.get(language, "en")  # Default to English if not found

# Routes --------------------------------




# Database setup function
def init_db():
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()

    # Create users table if it doesn't exist
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL
    )
    ''')

    conn.commit()
    conn.close()
    print("Database initialized successfully!")



@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        email = request.form['email']

        # Hash the password
        hashed_password = generate_password_hash(password)

        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()

        try:
            cursor.execute('INSERT INTO users (username, password, email) VALUES (?, ?, ?)',
                       (username, hashed_password, email))
            conn.commit()
            flash('Account created successfully! Please log in.', 'success')
            return redirect(url_for('signin'))
        except sqlite3.IntegrityError:
            flash('Username or email already exists!', 'error')
        finally:
            conn.close()

    return render_template('signup.html')

@app.route('/signin', methods=['GET', 'POST'])
def signin():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()

        cursor.execute('SELECT * FROM users WHERE username = ?', (username,))
        user = cursor.fetchone()
        conn.close()

        if user and check_password_hash(user[2], password):
            session['username'] = username
            flash('Logged in successfully!', 'success')
            return redirect(url_for('home'))
        else:
            flash('Invalid username or password!', 'error')

    return render_template('signin.html')

@app.route('/logout')
def logout():
    session.pop('username', None)
    flash('You have been logged out', 'info')
    return redirect(url_for('signin'))

# Generate authentication templates
def generate_auth_templates():
    # Signup template
    with open('templates/signup.html', 'w') as f:
        f.write(''' <!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>StellarSpeech - Sign Up</title>
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: 'Poppins', sans-serif;
        }

        body {
            min-height: 100vh;
            background: linear-gradient(135deg, #141e30, #243b55);
            color: #fff;
            overflow-x: hidden;
        }

        .navbar {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 70px;
            background: rgba(0, 0, 0, 0.2);
            backdrop-filter: blur(10px);
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 0 50px;
            z-index: 1000;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.1);
        }

        .navbar .logo {
            display: flex;
            align-items: center;
        }

        .navbar .logo .icon {
            font-size: 28px;
            margin-right: 8px;
            color: #7dabf8;
            animation: pulse 2s infinite;
        }

        .navbar .logo h1 {
            font-size: 22px;
            font-weight: 600;
            background: linear-gradient(45deg, #4facfe, #00f2fe);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            letter-spacing: 1px;
        }

        .navbar .menu {
            display: flex;
            gap: 30px;
        }

        .navbar .menu a {
            color: rgba(255, 255, 255, 0.8);
            text-decoration: none;
            font-weight: 500;
            font-size: 16px;
            transition: 0.3s;
            position: relative;
        }

        .navbar .menu a:hover {
            color: #fff;
        }

        .navbar .menu a::after {
            content: '';
            position: absolute;
            bottom: -5px;
            left: 0;
            width: 0;
            height: 2px;
            background: linear-gradient(45deg, #4facfe, #00f2fe);
            transition: width 0.3s;
        }

        .navbar .menu a:hover::after {
            width: 100%;
        }

        main {
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            padding-top: 70px;
            position: relative;
        }

        .container {
            width: 420px;
            padding: 40px;
            background: rgba(255, 255, 255, 0.05);
            box-shadow: 0 15px 35px rgba(0, 0, 0, 0.2);
            border-radius: 15px;
            backdrop-filter: blur(10px);
            border: 1px solid rgba(255, 255, 255, 0.1);
            position: relative;
            overflow: hidden;
            animation: fadeIn 1s ease-out;
            transition: transform 0.3s, box-shadow 0.3s;
        }

        .container:hover {
            transform: translateY(-5px);
            box-shadow: 0 20px 40px rgba(0, 0, 0, 0.3);
        }

        .container::before {
            content: '';
            position: absolute;
            top: -50%;
            left: -50%;
            width: 200%;
            height: 200%;
            background: radial-gradient(circle, rgba(255, 255, 255, 0.1) 0%, transparent 60%);
            transform: rotate(0deg);
            animation: rotate 15s linear infinite;
            pointer-events: none;
        }

        h2 {
            text-align: center;
            margin-bottom: 30px;
            color: #fff;
            font-weight: 600;
            position: relative;
            padding-bottom: 10px;
        }

        h2::after {
            content: '';
            position: absolute;
            bottom: 0;
            left: 50%;
            transform: translateX(-50%);
            width: 60px;
            height: 3px;
            background: linear-gradient(45deg, #4facfe, #00f2fe);
            border-radius: 3px;
        }

        .form-group {
            margin-bottom: 25px;
            position: relative;
            overflow: hidden;
        }

        .form-group i {
            position: absolute;
            top: 15px;
            left: 15px;
            color: #7dabf8;
            transition: 0.3s;
            z-index: 1;
        }

        .form-control {
            width: 100%;
            padding: 12px 20px 12px 45px;
            background: rgba(255, 255, 255, 0.1);
            border: none;
            border-radius: 35px;
            color: #fff;
            font-size: 16px;
            border: 1px solid rgba(255, 255, 255, 0.1);
            outline: none;
            transition: 0.3s;
        }

        .form-control:focus {
            border-color: #7dabf8;
            background: rgba(255, 255, 255, 0.2);
        }

        .form-control:focus + i {
            transform: translateX(-3px);
            color: #00f2fe;
        }

        .form-control::placeholder {
            color: rgba(255, 255, 255, 0.7);
        }

        .btn {
            width: 100%;
            padding: 14px 20px;
            background: linear-gradient(45deg, #4facfe, #00f2fe);
            border: none;
            border-radius: 35px;
            color: #fff;
            font-size: 16px;
            font-weight: 500;
            cursor: pointer;
            transition: 0.3s;
            position: relative;
            overflow: hidden;
            z-index: 1;
        }

        .btn::before {
            content: '';
            position: absolute;
            top: 0;
            left: -100%;
            width: 100%;
            height: 100%;
            background: linear-gradient(45deg, #00f2fe, #4facfe);
            transition: 0.5s;
            z-index: -1;
        }

        .btn:hover::before {
            left: 0;
        }

        .btn:hover {
            transform: translateY(-3px);
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.2);
        }

        .btn:active {
            transform: translateY(-1px);
            box-shadow: 0 5px 10px rgba(0, 0, 0, 0.2);
        }

        .link {
            text-align: center;
            margin-top: 25px;
            color: rgba(255, 255, 255, 0.7);
            font-size: 15px;
            opacity: 0;
            animation: fadeInUp 0.8s forwards;
            animation-delay: 0.6s;
        }

        .link a {
            color: #7dabf8;
            text-decoration: none;
            font-weight: 500;
            transition: 0.3s;
            position: relative;
        }

        .link a:hover {
            color: #00f2fe;
        }

        .link a::after {
            content: '';
            position: absolute;
            bottom: -2px;
            left: 0;
            width: 0;
            height: 1px;
            background: #00f2fe;
            transition: width 0.3s;
        }

        .link a:hover::after {
            width: 100%;
        }

        .alert {
            padding: 12px 15px;
            margin-bottom: 25px;
            border-radius: 8px;
            color: #fff;
            font-size: 14px;
            animation: slideInDown 0.5s forwards;
            display: flex;
            align-items: center;
        }

        .alert i {
            margin-right: 10px;
            font-size: 16px;
        }

        .alert-success {
            background-color: rgba(40, 167, 69, 0.7);
            border-left: 4px solid #28a745;
        }

        .alert-danger {
            background-color: rgba(220, 53, 69, 0.7);
            border-left: 4px solid #dc3545;
        }

        /* Animation for stars in background */
        .stars {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            pointer-events: none;
        }

        .star {
            position: absolute;
            background: rgba(255, 255, 255, 0.5);
            border-radius: 50%;
            animation: twinkle var(--duration) infinite;
            opacity: 0;
        }

        /* Animations */
        @keyframes fadeIn {
            from {
                opacity: 0;
                transform: translateY(20px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        @keyframes fadeInUp {
            from {
                opacity: 0;
                transform: translateY(10px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        @keyframes slideInDown {
            from {
                opacity: 0;
                transform: translateY(-20px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        @keyframes pulse {
            0% {
                transform: scale(1);
            }
            50% {
                transform: scale(1.1);
            }
            100% {
                transform: scale(1);
            }
        }

        @keyframes rotate {
            0% {
                transform: rotate(0deg);
            }
            100% {
                transform: rotate(360deg);
            }
        }

        @keyframes twinkle {
            0% {
                opacity: 0;
                transform: scale(0.5);
            }
            50% {
                opacity: 1;
                transform: scale(1);
            }
            100% {
                opacity: 0;
                transform: scale(0.5);
            }
        }

        /* Form animation */
        .form-group {
            opacity: 0;
            transform: translateY(20px);
            animation: fadeInUp 0.8s forwards;
        }

        .form-group:nth-child(1) {
            animation-delay: 0.2s;
        }

        .form-group:nth-child(2) {
            animation-delay: 0.3s;
        }

        .form-group:nth-child(3) {
            animation-delay: 0.4s;
        }

        button.btn {
            opacity: 0;
            animation: fadeInUp 0.8s forwards;
            animation-delay: 0.5s;
        }

        /* Responsive */
        @media (max-width: 768px) {
            .navbar {
                padding: 0 20px;
            }

            .navbar .menu {
                gap: 15px;
            }

            .container {
                width: 90%;
                max-width: 400px;
            }
        }
    </style>
</head>
<body>
    <nav class="navbar">
        <div class="logo">
            <i class="fas fa-comment-dots icon"></i>
            <h1>StellarSpeech</h1>
        </div>
    </nav>

    <main>
        <div class="container">
            <h2>Create Account</h2>

            {% with messages = get_flashed_messages(with_categories=true) %}
                {% if messages %}
                    {% for category, message in messages %}
                        <div class="alert alert-{{ 'success' if category == 'success' else 'danger' }}">
                            <i class="{{ 'fas fa-check-circle' if category == 'success' else 'fas fa-exclamation-circle' }}"></i>
                            {{ message }}
                        </div>
                    {% endfor %}
                {% endif %}
            {% endwith %}

            <form method="POST" action="/signup">
                <div class="form-group">
                    <input type="text" class="form-control" id="username" name="username" placeholder="Username" required>
                    <i class="fas fa-user"></i>
                </div>
                <div class="form-group">
                    <input type="email" class="form-control" id="email" name="email" placeholder="Email" required>
                    <i class="fas fa-envelope"></i>
                </div>
                <div class="form-group">
                    <input type="password" class="form-control" id="password" name="password" placeholder="Password" required>
                    <i class="fas fa-lock"></i>
                </div>
                <button type="submit" class="btn">Sign Up</button>
            </form>
            <div class="link">
                Already have an account? <a href="/signin">Sign In</a>
            </div>
        </div>
    </main>

    <div class="stars" id="stars"></div>

    <script>
        // Create animated stars in the background
        document.addEventListener('DOMContentLoaded', () => {
            const starsContainer = document.getElementById('stars');
            const numberOfStars = 70;

            for (let i = 0; i < numberOfStars; i++) {
                const star = document.createElement('div');
                star.classList.add('star');

                // Random position
                const x = Math.random() * 100;
                const y = Math.random() * 100;

                // Random size
                const size = Math.random() * 3;

                // Random duration
                const duration = 2 + Math.random() * 3;

                // Random delay
                const delay = Math.random() * 5;

                star.style.left = `${x}%`;
                star.style.top = `${y}%`;
                star.style.width = `${size}px`;
                star.style.height = `${size}px`;
                star.style.setProperty('--duration', `${duration}s`);
                star.style.animationDelay = `${delay}s`;

                starsContainer.appendChild(star);
            }

            // Form interaction effects
            const formControls = document.querySelectorAll('.form-control');
            formControls.forEach(input => {
                input.addEventListener('focus', () => {
                    input.parentElement.style.transform = 'translateX(5px)';
                    setTimeout(() => {
                        input.parentElement.style.transform = 'translateX(0)';
                    }, 300);
                });
            });

            // Container hover effect
            const container = document.querySelector('.container');
            container.addEventListener('mousemove', (e) => {
                const x = e.clientX - container.getBoundingClientRect().left;
                const y = e.clientY - container.getBoundingClientRect().top;

                const centerX = container.offsetWidth / 2;
                const centerY = container.offsetHeight / 2;

                const moveX = (x - centerX) / 30;
                const moveY = (y - centerY) / 30;

                container.style.transform = `translateY(-5px) rotateX(${-moveY}deg) rotateY(${moveX}deg)`;
            });

            container.addEventListener('mouseleave', () => {
                container.style.transform = 'translateY(-5px) rotateX(0deg) rotateY(0deg)';
            });
        });
    </script>
</body>
</html> ''')

    # Signin template
    with open('templates/signin.html', 'w') as f:
        f.write(''' <!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>StellarSpeech - Sign In</title>
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: 'Poppins', sans-serif;
        }

        body {
            min-height: 100vh;
            background: linear-gradient(135deg, #141e30, #243b55);
            color: #fff;
            overflow-x: hidden;
        }

        .navbar {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 70px;
            background: rgba(0, 0, 0, 0.2);
            backdrop-filter: blur(10px);
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 0 50px;
            z-index: 1000;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.1);
        }

        .navbar .logo {
            display: flex;
            align-items: center;
        }

        .navbar .logo .icon {
            font-size: 28px;
            margin-right: 8px;
            color: #7dabf8;
            animation: pulse 2s infinite;
        }

        .navbar .logo h1 {
            font-size: 22px;
            font-weight: 600;
            background: linear-gradient(45deg, #4facfe, #00f2fe);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            letter-spacing: 1px;
        }

        .navbar .menu {
            display: flex;
            gap: 30px;
        }

        .navbar .menu a {
            color: rgba(255, 255, 255, 0.8);
            text-decoration: none;
            font-weight: 500;
            font-size: 16px;
            transition: 0.3s;
            position: relative;
        }

        .navbar .menu a:hover {
            color: #fff;
        }

        .navbar .menu a::after {
            content: '';
            position: absolute;
            bottom: -5px;
            left: 0;
            width: 0;
            height: 2px;
            background: linear-gradient(45deg, #4facfe, #00f2fe);
            transition: width 0.3s;
        }

        .navbar .menu a:hover::after {
            width: 100%;
        }

        main {
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            padding-top: 70px;
            position: relative;
        }

        .container {
            width: 420px;
            padding: 40px;
            background: rgba(255, 255, 255, 0.05);
            box-shadow: 0 15px 35px rgba(0, 0, 0, 0.2);
            border-radius: 15px;
            backdrop-filter: blur(10px);
            border: 1px solid rgba(255, 255, 255, 0.1);
            position: relative;
            overflow: hidden;
            animation: fadeIn 1s ease-out;
            transition: transform 0.3s, box-shadow 0.3s;
        }

        .container:hover {
            transform: translateY(-5px);
            box-shadow: 0 20px 40px rgba(0, 0, 0, 0.3);
        }

        .container::before {
            content: '';
            position: absolute;
            top: -50%;
            left: -50%;
            width: 200%;
            height: 200%;
            background: radial-gradient(circle, rgba(255, 255, 255, 0.1) 0%, transparent 60%);
            transform: rotate(0deg);
            animation: rotate 15s linear infinite;
            pointer-events: none;
        }

        h2 {
            text-align: center;
            margin-bottom: 30px;
            color: #fff;
            font-weight: 600;
            position: relative;
            padding-bottom: 10px;
        }

        h2::after {
            content: '';
            position: absolute;
            bottom: 0;
            left: 50%;
            transform: translateX(-50%);
            width: 60px;
            height: 3px;
            background: linear-gradient(45deg, #4facfe, #00f2fe);
            border-radius: 3px;
        }

        .form-group {
            margin-bottom: 25px;
            position: relative;
            overflow: hidden;
        }

        .form-group i {
            position: absolute;
            top: 15px;
            left: 15px;
            color: #7dabf8;
            transition: 0.3s;
            z-index: 1;
        }

        .form-control {
            width: 100%;
            padding: 12px 20px 12px 45px;
            background: rgba(255, 255, 255, 0.1);
            border: none;
            border-radius: 35px;
            color: #fff;
            font-size: 16px;
            border: 1px solid rgba(255, 255, 255, 0.1);
            outline: none;
            transition: 0.3s;
        }

        .form-control:focus {
            border-color: #7dabf8;
            background: rgba(255, 255, 255, 0.2);
        }

        .form-control:focus + i {
            transform: translateX(-3px);
            color: #00f2fe;
        }

        .form-control::placeholder {
            color: rgba(255, 255, 255, 0.7);
        }

        .btn {
            width: 100%;
            padding: 14px 20px;
            background: linear-gradient(45deg, #4facfe, #00f2fe);
            border: none;
            border-radius: 35px;
            color: #fff;
            font-size: 16px;
            font-weight: 500;
            cursor: pointer;
            transition: 0.3s;
            position: relative;
            overflow: hidden;
            z-index: 1;
        }

        .btn::before {
            content: '';
            position: absolute;
            top: 0;
            left: -100%;
            width: 100%;
            height: 100%;
            background: linear-gradient(45deg, #00f2fe, #4facfe);
            transition: 0.5s;
            z-index: -1;
        }

        .btn:hover::before {
            left: 0;
        }

        .btn:hover {
            transform: translateY(-3px);
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.2);
        }

        .btn:active {
            transform: translateY(-1px);
            box-shadow: 0 5px 10px rgba(0, 0, 0, 0.2);
        }

        .link {
            text-align: center;
            margin-top: 25px;
            color: rgba(255, 255, 255, 0.7);
            font-size: 15px;
            opacity: 0;
            animation: fadeInUp 0.8s forwards;
            animation-delay: 0.5s;
        }

        .link a {
            color: #7dabf8;
            text-decoration: none;
            font-weight: 500;
            transition: 0.3s;
            position: relative;
        }

        .link a:hover {
            color: #00f2fe;
        }

        .link a::after {
            content: '';
            position: absolute;
            bottom: -2px;
            left: 0;
            width: 0;
            height: 1px;
            background: #00f2fe;
            transition: width 0.3s;
        }

        .link a:hover::after {
            width: 100%;
        }

        .alert {
            padding: 12px 15px;
            margin-bottom: 25px;
            border-radius: 8px;
            color: #fff;
            font-size: 14px;
            animation: slideInDown 0.5s forwards;
            display: flex;
            align-items: center;
        }

        .alert i {
            margin-right: 10px;
            font-size: 16px;
        }

        .alert-success {
            background-color: rgba(40, 167, 69, 0.7);
            border-left: 4px solid #28a745;
        }

        .alert-danger {
            background-color: rgba(220, 53, 69, 0.7);
            border-left: 4px solid #dc3545;
        }

        /* Animation for stars in background */
        .stars {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            pointer-events: none;
        }

        .star {
            position: absolute;
            background: rgba(255, 255, 255, 0.5);
            border-radius: 50%;
            animation: twinkle var(--duration) infinite;
            opacity: 0;
        }

        /* Animation for meteor effect */
        .meteor {
            position: absolute;
            width: 2px;
            height: 60px;
            background: linear-gradient(to bottom, rgba(255, 255, 255, 0), rgba(255, 255, 255, 0.7));
            transform: rotate(45deg);
            pointer-events: none;
            opacity: 0;
            filter: blur(1px);
        }

        /* Animations */
        @keyframes fadeIn {
            from {
                opacity: 0;
                transform: translateY(20px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        @keyframes fadeInUp {
            from {
                opacity: 0;
                transform: translateY(10px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        @keyframes slideInDown {
            from {
                opacity: 0;
                transform: translateY(-20px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        @keyframes pulse {
            0% {
                transform: scale(1);
            }
            50% {
                transform: scale(1.1);
            }
            100% {
                transform: scale(1);
            }
        }

        @keyframes rotate {
            0% {
                transform: rotate(0deg);
            }
            100% {
                transform: rotate(360deg);
            }
        }

        @keyframes twinkle {
            0% {
                opacity: 0;
                transform: scale(0.5);
            }
            50% {
                opacity: 1;
                transform: scale(1);
            }
            100% {
                opacity: 0;
                transform: scale(0.5);
            }
        }

        @keyframes meteorFall {
            0% {
                transform: translate(0, 0) rotate(45deg);
                opacity: 1;
            }
            100% {
                transform: translate(300px, 300px) rotate(45deg);
                opacity: 0;
            }
        }

        /* Form animation */
        .form-group {
            opacity: 0;
            transform: translateY(20px);
            animation: fadeInUp 0.8s forwards;
        }

        .form-group:nth-child(1) {
            animation-delay: 0.2s;
        }

        .form-group:nth-child(2) {
            animation-delay: 0.3s;
        }

        button.btn {
            opacity: 0;
            animation: fadeInUp 0.8s forwards;
            animation-delay: 0.4s;
        }

        /* Responsive */
        @media (max-width: 768px) {
            .navbar {
                padding: 0 20px;
            }

            .navbar .menu {
                gap: 15px;
            }

            .container {
                width: 90%;
                max-width: 400px;
            }
        }
    </style>
</head>
<body>
    <nav class="navbar">
        <div class="logo">
            <i class="fas fa-comment-dots icon"></i>
            <h1>StellarSpeech</h1>
        </div>

    </nav>

    <main>
        <div class="container">
            <h2>Sign In</h2>

            {% with messages = get_flashed_messages(with_categories=true) %}
                {% if messages %}
                    {% for category, message in messages %}
                        <div class="alert alert-{{ 'success' if category == 'success' else 'danger' }}">
                            <i class="{{ 'fas fa-check-circle' if category == 'success' else 'fas fa-exclamation-circle' }}"></i>
                            {{ message }}
                        </div>
                    {% endfor %}
                {% endif %}
            {% endwith %}

            <form method="POST" action="/signin">
                <div class="form-group">
                    <input type="text" class="form-control" id="username" name="username" placeholder="Username" required>
                    <i class="fas fa-user"></i>
                </div>
                <div class="form-group">
                    <input type="password" class="form-control" id="password" name="password" placeholder="Password" required>
                    <i class="fas fa-lock"></i>
                </div>
                <button type="submit" class="btn">Sign In</button>
            </form>
            <div class="link">
                Don't have an account? <a href="/signup">Sign Up</a>
            </div>
        </div>
    </main>

    <div class="stars" id="stars"></div>

    <script>
        // Create animated stars in the background
        document.addEventListener('DOMContentLoaded', () => {
            const starsContainer = document.getElementById('stars');
            const numberOfStars = 70;

            for (let i = 0; i < numberOfStars; i++) {
                const star = document.createElement('div');
                star.classList.add('star');

                // Random position
                const x = Math.random() * 100;
                const y = Math.random() * 100;

                // Random size
                const size = Math.random() * 3;

                // Random duration
                const duration = 2 + Math.random() * 3;

                // Random delay
                const delay = Math.random() * 5;

                star.style.left = `${x}%`;
                star.style.top = `${y}%`;
                star.style.width = `${size}px`;
                star.style.height = `${size}px`;
                star.style.setProperty('--duration', `${duration}s`);
                star.style.animationDelay = `${delay}s`;

                starsContainer.appendChild(star);
            }

            // Create occasional meteor animation
            const createMeteor = () => {
                const meteor = document.createElement('div');
                meteor.classList.add('meteor');

                const x = Math.random() * 100;
                const y = Math.random() * 20;

                meteor.style.left = `${x}%`;
                meteor.style.top = `${y}%`;
                meteor.style.animation = `meteorFall ${1 + Math.random() * 2}s linear forwards`;

                starsContainer.appendChild(meteor);

                // Remove meteor after animation completes
                setTimeout(() => {
                    meteor.remove();
                }, 3000);
            };

            // Create a meteor every few seconds
            setInterval(createMeteor, 5000);

            // Form interaction effects
            const formControls = document.querySelectorAll('.form-control');
            formControls.forEach(input => {
                input.addEventListener('focus', () => {
                    input.parentElement.style.transform = 'translateX(5px)';
                    setTimeout(() => {
                        input.parentElement.style.transform = 'translateX(0)';
                    }, 300);
                });
            });

            // Container hover effect
            const container = document.querySelector('.container');
            container.addEventListener('mousemove', (e) => {
                const x = e.clientX - container.getBoundingClientRect().left;
                const y = e.clientY - container.getBoundingClientRect().top;

                const centerX = container.offsetWidth / 2;
                const centerY = container.offsetHeight / 2;

                const moveX = (x - centerX) / 30;
                const moveY = (y - centerY) / 30;

                container.style.transform = `translateY(-5px) rotateX(${-moveY}deg) rotateY(${moveX}deg)`;
            });

            container.addEventListener('mouseleave', () => {
                container.style.transform = 'translateY(-5px) rotateX(0deg) rotateY(0deg)';
            });
        });
    </script>
</body>
</html>''')

    return "Authentication templates generated successfully!"






# 1. Add settings route for password update

@app.route('/settings', methods=['GET', 'POST'])
def settings():
    # Protected route - only for authenticated users
    if 'username' not in session:
        return redirect(url_for('signin'))

    username = session['username']
    success_message = None
    error_message = None

    if request.method == 'POST':
        current_password = request.form['current_password']
        new_password = request.form['new_password']
        confirm_password = request.form['confirm_password']

        # Validation checks
        if new_password != confirm_password:
            error_message = "New passwords don't match!"
        else:
            # Get current user data
            conn = sqlite3.connect('users.db')
            cursor = conn.cursor()
            cursor.execute('SELECT * FROM users WHERE username = ?', (username,))
            user = cursor.fetchone()

            # Verify current password
            if user and check_password_hash(user[2], current_password):
                # Update password
                hashed_new_password = generate_password_hash(new_password)
                cursor.execute('UPDATE users SET password = ? WHERE username = ?',
                              (hashed_new_password, username))
                conn.commit()
                success_message = "Password updated successfully!"
            else:
                error_message = "Current password is incorrect!"

            conn.close()

    return render_template('settings.html', username=username,
                           success_message=success_message,
                           error_message=error_message)

@app.route('/')
def home():
    # Protected route - only for authenticated users
    if 'username' not in session:
        return redirect(url_for('signin'))

    # Pass username to the template
    username = session['username']
    return render_template('index.html', username=username)

# 3. Generate settings template function
def generate_settings_template():
    with open('templates/settings.html', 'w') as f:
        f.write('''<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>StellarSpeech - Settings</title>
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: 'Poppins', sans-serif;
        }
        :root {
            --primary-color: #4f46e5;
            --primary-dark: #4338ca;
            --secondary-color: #10b981;
            --text-color: #374151;
            --light-bg: #f9fafb;
            --card-bg: #ffffff;
            --border-color: #e5e7eb;
            --error-color: #ef4444;
            --success-color: #10b981;
        }
        body {
            font-family: 'Poppins', sans-serif;
            background-color: var(--light-bg);
            color: var(--text-color);
            line-height: 1.6;
        }
        /* Navigation Bar Styles */
        .navbar {
            background-color: var(--card-bg);
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            padding: 0.8rem 2rem;
            position: sticky;
            top: 0;
            z-index: 100;
        }
        .nav-container {
            max-width: 1200px;
            margin: 0 auto;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }
        .nav-logo {
            display: flex;
            align-items: center;
            text-decoration: none;
        }
        .nav-logo i {
            font-size: 1.8rem;
            color: var(--primary-color);
            margin-right: 0.5rem;
        }
        .nav-logo h2 {
            color: var(--primary-color);
            font-size: 1.5rem;
            font-weight: 600;
        }
        .nav-links {
            display: flex;
            align-items: center;
        }
        .nav-links a {
            color: var(--text-color);
            margin-left: 1.5rem;
            text-decoration: none;
            font-weight: 500;
            transition: color 0.3s ease;
        }
        .nav-links a:hover {
            color: var(--primary-color);
        }
        .nav-links .user-greeting {
            margin-right: 1rem;
            font-weight: 500;
        }
        .nav-links .logout-btn {
            background-color: var(--primary-color);
            color: white;
            border: none;
            border-radius: 5px;
            padding: 0.5rem 1rem;
            font-size: 0.9rem;
            font-weight: 500;
            cursor: pointer;
            transition: background-color 0.3s ease;
            display: flex;
            align-items: center;
            font-family: 'Poppins', sans-serif;
        }
        .nav-links .logout-btn i {
            margin-right: 0.4rem;
        }
        .logout-btn:hover {
            background-color: var(--primary-dark);
        }
        .container {
            max-width: 800px;
            margin: 2rem auto;
            padding: 0 1rem;
        }
        .settings-card {
            background-color: var(--card-bg);
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            padding: 2rem;
        }
        .settings-header {
            margin-bottom: 1.5rem;
            display: flex;
            align-items: center;
        }
        .settings-header i {
            font-size: 1.8rem;
            color: var(--primary-color);
            margin-right: 0.8rem;
        }
        .settings-header h2 {
            font-size: 1.8rem;
            color: var(--primary-color);
        }
        .form-group {
            margin-bottom: 1.5rem;
        }
        .form-group label {
            display: block;
            margin-bottom: 0.5rem;
            font-weight: 500;
        }
        .form-control {
            width: 100%;
            padding: 0.75rem;
            border: 1px solid var(--border-color);
            border-radius: 5px;
            font-size: 1rem;
            transition: border-color 0.3s;
        }
        .form-control:focus {
            outline: none;
            border-color: var(--primary-color);
        }
        .btn-update {
            background-color: var(--primary-color);
            color: white;
            border: none;
            border-radius: 5px;
            padding: 0.75rem 1.5rem;
            font-size: 1rem;
            font-weight: 500;
            cursor: pointer;
            transition: background-color 0.3s;
            display: inline-flex;
            align-items: center;
        }
        .btn-update i {
            margin-right: 0.5rem;
        }
        .btn-update:hover {
            background-color: var(--primary-dark);
        }
        .alert {
            padding: 1rem;
            border-radius: 5px;
            margin-bottom: 1.5rem;
        }
        .alert-success {
            background-color: rgba(16, 185, 129, 0.1);
            border: 1px solid var(--success-color);
            color: var(--success-color);
        }
        .alert-error {
            background-color: rgba(239, 68, 68, 0.1);
            border: 1px solid var(--error-color);
            color: var(--error-color);
        }
        footer {
            text-align: center;
            margin-top: 3rem;
            color: var(--text-color);
            opacity: 0.8;
            font-size: 0.9rem;
        }
    </style>
</head>
<body>
    <!-- Navigation Bar -->
    <nav class="navbar">
        <div class="nav-container">
            <a href="/" class="nav-logo">
                <i class="fas fa-microphone-alt"></i>
                <h2>StellarSpeech</h2>
            </a>
            <div class="nav-links">
                <span class="user-greeting">Hello, {{ username }}!</span>
                <a href="/">Dashboard</a>
                <a href="/settings" class="active">Settings</a>
                <a href="/logout" class="logout-btn">
                    <i class="fas fa-sign-out-alt"></i> Logout
                </a>
            </div>
        </div>
    </nav>

    <div class="container">
        <div class="settings-card">
            <div class="settings-header">
                <i class="fas fa-cog"></i>
                <h2>Account Settings</h2>
            </div>

            {% if success_message %}
            <div class="alert alert-success">
                <i class="fas fa-check-circle"></i> {{ success_message }}
            </div>
            {% endif %}

            {% if error_message %}
            <div class="alert alert-error">
                <i class="fas fa-exclamation-circle"></i> {{ error_message }}
            </div>
            {% endif %}

            <form method="POST" action="/settings">
                <div class="form-group">
                    <label for="username">Username</label>
                    <input type="text" class="form-control" id="username" value="{{ username }}" disabled>
                </div>
                <div class="form-group">
                    <label for="current_password">Current Password</label>
                    <input type="password" class="form-control" id="current_password" name="current_password" required>
                </div>
                <div class="form-group">
                    <label for="new_password">New Password</label>
                    <input type="password" class="form-control" id="new_password" name="new_password" required>
                </div>
                <div class="form-group">
                    <label for="confirm_password">Confirm New Password</label>
                    <input type="password" class="form-control" id="confirm_password" name="confirm_password" required>
                </div>
                <button type="submit" class="btn-update">
                    <i class="fas fa-save"></i> Update Password
                </button>
            </form>
        </div>
    </div>

    <footer>
        <p>&copy; 2025 StellarSpeech. All rights reserved.</p>
    </footer>
</body>
</html>''')
    return "Settings template generated successfully!"





@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    language = request.form.get('language', 'auto')  # Get language parameter or default to auto

    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)

        try:
            # Process the audio file
            emotion = predict_emotion(file_path)

            # Variables to store results
            transcription = ""
            translation = ""
            detected_language = ""
            src_lang_code = ""
            tgt_lang_code = ""

            # Transcribe based on the specified language or auto-detect
            if language == 'hindi' or language == 'auto':
                hindi_transcription = transcribe_audio_hindi(file_path)
                hindi_to_english = translate_hindi_to_english(hindi_transcription)

                # For auto mode, check if the transcription looks like Hindi
                if language == 'auto' and detect_language(hindi_transcription) == 'hindi':
                    transcription = hindi_transcription
                    translation = hindi_to_english
                    detected_language = 'hindi'
                    src_lang_code = 'hi'
                    tgt_lang_code = 'en'
                elif language == 'hindi':
                    transcription = hindi_transcription
                    translation = hindi_to_english
                    detected_language = 'hindi'
                    src_lang_code = 'hi'
                    tgt_lang_code = 'en'

            if (language == 'telugu' or language == 'auto') and not detected_language:
                telugu_transcription = transcribe_audio_telugu(file_path)
                telugu_to_english = translate_telugu_to_english(telugu_transcription)

                # For auto mode, check if the transcription looks like Telugu
                if language == 'auto' and detect_language(telugu_transcription) == 'telugu':
                    transcription = telugu_transcription
                    translation = telugu_to_english
                    detected_language = 'telugu'
                    src_lang_code = 'te'
                    tgt_lang_code = 'en'
                elif language == 'telugu':
                    transcription = telugu_transcription
                    translation = telugu_to_english
                    detected_language = 'telugu'
                    src_lang_code = 'te'
                    tgt_lang_code = 'en'

            if (language == 'english' or language == 'auto') and not detected_language:
                english_transcription = transcribe_audio_english(file_path)
                english_to_hindi = translate_english_to_hindi(english_transcription)

                # English is the default for auto mode if no other script is detected
                transcription = english_transcription
                translation = english_to_hindi
                detected_language = 'english'
                src_lang_code = 'en'
                tgt_lang_code = 'hi'

            # Generate speech from the transcription and translation
            transcription_speech_path = None
            translation_speech_path = None

            # Generate speech for transcription (source language)
            if transcription:
                transcription_filename = f"transcription_{detected_language}_{int(os.path.getmtime(file_path))}.mp3"
                transcription_speech_path = os.path.join(app.config['TTS_FOLDER'], transcription_filename)
                transcription_speech_url = generate_speech(transcription, src_lang_code, transcription_speech_path)

            # Generate speech for translation (target language)
            if translation:
                translation_filename = f"translation_{tgt_lang_code}_{int(os.path.getmtime(file_path))}.mp3"
                translation_speech_path = os.path.join(app.config['TTS_FOLDER'], translation_filename)
                translation_speech_url = generate_speech(translation, tgt_lang_code, translation_speech_path)

            # Return the results with audio URLs
            return jsonify({
                'emotion': emotion,
                'transcription': transcription,
                'translation': translation,
                'detected_language': detected_language,
                'transcription_audio': f'/get_audio?file={os.path.basename(transcription_speech_path)}' if transcription_speech_path else None,
                'translation_audio': f'/get_audio?file={os.path.basename(translation_speech_path)}' if translation_speech_path else None
            })

        except Exception as e:
            return jsonify({'error': str(e)}), 500

    return jsonify({'error': 'File type not allowed'}), 400

# Route to serve the generated audio files
@app.route('/get_audio')
def get_audio():
    filename = request.args.get('file')
    if not filename:
        return jsonify({'error': 'No filename provided'}), 400

    file_path = os.path.join(app.config['TTS_FOLDER'], filename)
    if not os.path.exists(file_path):
        return jsonify({'error': 'Audio file not found'}), 404

    return send_file(file_path, mimetype="audio/mpeg")

# HTML Template
@app.route('/generate_template')
def generate_template():
    html = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>StellarSpeech: Real-Time Voice Translator</title>
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">
    <style>
        :root {
            --primary-color: #4f46e5;
            --primary-dark: #4338ca;
            --secondary-color: #10b981;
            --text-color: #374151;
            --light-bg: #f9fafb;
            --card-bg: #ffffff;
            --border-color: #e5e7eb;
            --error-color: #ef4444;
        }

        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Poppins', sans-serif;
            background-color: var(--light-bg);
            color: var(--text-color);
            line-height: 1.6;
        }

        /* Navigation Bar Styles */
        .navbar {
            background-color: var(--card-bg);
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            padding: 0.8rem 2rem;
            position: sticky;
            top: 0;
            z-index: 100;
        }

        .nav-container {
            max-width: 1200px;
            margin: 0 auto;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }

        .nav-logo {
            display: flex;
            align-items: center;
            text-decoration: none;
        }

        .nav-logo i {
            font-size: 1.8rem;
            color: var(--primary-color);
            margin-right: 0.5rem;
        }

        .nav-logo h2 {
            color: var(--primary-color);
            font-size: 1.5rem;
            font-weight: 600;
        }

        .nav-links {
            display: flex;
            align-items: center;
        }

        .nav-links a {
            color: var(--text-color);
            margin-left: 1.5rem;
            text-decoration: none;
            font-weight: 500;
            transition: color 0.3s ease;
        }

        .nav-links a:hover {
            color: var(--primary-color);
        }

        .nav-links .logout-btn {
            background-color: var(--primary-color);
            color: white;
            border: none;
            border-radius: 5px;
            padding: 0.5rem 1rem;
            font-size: 0.9rem;
            font-weight: 500;
            cursor: pointer;
            transition: background-color 0.3s ease;
            display: flex;
            align-items: center;
            font-family: 'Poppins', sans-serif;
        }

        .nav-links .logout-btn i {
            margin-right: 0.4rem;
        }

        .logout-btn:hover {
            background-color: var(--primary-dark);
        }

        /* Media query for responsive navbar */
        @media (max-width: 768px) {
            .navbar {
                padding: 0.8rem 1rem;
            }

            .nav-logo h2 {
                font-size: 1.3rem;
            }

            .nav-links a {
                margin-left: 1rem;
            }
        }

        /* Existing styles */
        .container {
            max-width: 1000px;
            margin: 0 auto;
            padding: 2rem 1rem;
        }

        header {
            text-align: center;
            margin-bottom: 2rem;
        }

        .logo {
            display: flex;
            align-items: center;
            justify-content: center;
            margin-bottom: 1rem;
        }

        .logo i {
            font-size: 2.5rem;
            color: var(--primary-color);
            margin-right: 0.5rem;
        }

        h1 {
            font-size: 2.5rem;
            color: var(--primary-color);
            margin-bottom: 0.5rem;
        }

        .slogan {
            font-size: 1.2rem;
            color: var(--text-color);
            opacity: 0.8;
        }

        .main-content {
            display: grid;
            grid-template-columns: 1fr;
            gap: 2rem;
        }

        @media (min-width: 768px) {
            .main-content {
                grid-template-columns: 1fr 1fr;
            }
        }

        .card {
            background-color: var(--card-bg);
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            padding: 1.5rem;
            transition: transform 0.3s ease, box-shadow 0.3s ease;
        }

        .card:hover {
            transform: translateY(-5px);
            box-shadow: 0 10px 15px rgba(0, 0, 0, 0.1);
        }

        .card h2 {
            font-size: 1.5rem;
            margin-bottom: 1rem;
            color: var(--primary-color);
            display: flex;
            align-items: center;
        }

        .card h2 i {
            margin-right: 0.5rem;
        }

        .form-group {
            margin-bottom: 1.5rem;
        }

        label {
            display: block;
            margin-bottom: 0.5rem;
            font-weight: 500;
        }

        .language-select {
            width: 100%;
            padding: 0.75rem;
            border: 1px solid var(--border-color);
            border-radius: 5px;
            font-family: 'Poppins', sans-serif;
            font-size: 1rem;
        }

        .file-upload {
            width: 100%;
            padding: 0.75rem;
            border: 2px dashed var(--border-color);
            border-radius: 5px;
            text-align: center;
            cursor: pointer;
            transition: border-color 0.3s ease;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            min-height: 150px;
        }

        .file-upload:hover {
            border-color: var(--primary-color);
        }

        .file-upload i {
            font-size: 2.5rem;
            color: var(--primary-color);
            margin-bottom: 0.5rem;
        }

        .file-upload-input {
            display: none;
        }

        .upload-btn, .record-btn, .stop-btn {
            background-color: var(--primary-color);
            color: white;
            border: none;
            border-radius: 5px;
            padding: 0.75rem 1.5rem;
            font-size: 1rem;
            font-weight: 500;
            cursor: pointer;
            transition: background-color 0.3s ease;
            display: flex;
            align-items: center;
            margin-right: 0.5rem;
        }

        .upload-btn:hover, .record-btn:hover, .stop-btn:hover {
            background-color: var(--primary-dark);
        }

        .upload-btn i, .record-btn i, .stop-btn i {
            margin-right: 0.5rem;
        }

        .stop-btn {
            background-color: var(--error-color);
        }

        .button-group {
            display: flex;
            justify-content: flex-start;
            margin-top: 1rem;
        }

        .result-card {
            border-radius: 10px;
            background-color: var(--card-bg);
            border-left: 5px solid var(--secondary-color);
            padding: 1.5rem;
            margin-bottom: 1.5rem;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        }

        .result-card h3 {
            display: flex;
            align-items: center;
            margin-bottom: 0.5rem;
            color: var(--primary-color);
        }

        .result-card h3 i {
            margin-right: 0.5rem;
        }

        .result-text {
            margin-bottom: 1rem;
            font-size: 1.1rem;
            line-height: 1.6;
        }

        .audio-player {
            display: flex;
            align-items: center;
            background-color: #f3f4f6;
            border-radius: 5px;
            padding: 0.5rem;
        }

        .audio-player audio {
            flex-grow: 1;
        }

        .emotion-tag {
            display: inline-flex;
            align-items: center;
            background-color: #818cf8;
            color: white;
            padding: 0.25rem 0.75rem;
            border-radius: 15px;
            font-size: 0.875rem;
            margin-bottom: 1rem;
        }

        .emotion-tag i {
            margin-right: 0.3rem;
        }

        .language-tag {
            display: inline-flex;
            align-items: center;
            background-color: #10b981;
            color: white;
            padding: 0.25rem 0.75rem;
            border-radius: 15px;
            font-size: 0.875rem;
            margin-left: 0.5rem;
        }

        .loader {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100px;
        }

        .spinner {
            border: 5px solid rgba(0, 0, 0, 0.1);
            width: 50px;
            height: 50px;
            border-radius: 50%;
            border-left-color: var(--primary-color);
            animation: spin 1s linear infinite;
        }

        @keyframes spin {
            0% {
                transform: rotate(0deg);
            }
            100% {
                transform: rotate(360deg);
            }
        }

        .error-message {
            color: var(--error-color);
            text-align: center;
            margin: 1rem 0;
        }

        footer {
            text-align: center;
            margin-top: 3rem;
            color: var(--text-color);
            opacity: 0.8;
            font-size: 0.9rem;
        }

        .hidden {
            display: none;
        }

        #recordingIndicator {
            display: flex;
            align-items: center;
            margin-top: 0.5rem;
            color: var(--error-color);
            animation: pulse 1.5s infinite;
        }

        @keyframes pulse {
            0% {
                opacity: 1;
            }
            50% {
                opacity: 0.5;
            }
            100% {
                opacity: 1;
            }
        }

        #recordingIndicator i {
            margin-right: 0.5rem;
        }

        .file-info {
            margin-top: 0.5rem;
            font-size: 0.9rem;
            color: var(--text-color);
            opacity: 0.8;
        }
    </style>
</head>
<body>
    <!-- Navigation Bar -->
    <nav class="navbar">
        <div class="nav-container">
            <a href="#" class="nav-logo">
                <i class="fas fa-microphone-alt"></i>
                <h2>StellarSpeech</h2>
            </a>
           <div class="nav-links">
    <span class="user-greeting">Hello, {{ username }}!</span>
    <a href="/" class="active">Dashboard</a>
    <a href="/settings">Settings</a>
    <a href="/logout" class="logout-btn">
        <i class="fas fa-sign-out-alt"></i> Logout
    </a>
</div>
        </div>
    </nav>

    <div class="container">
        <header>
            <div class="logo">
                <i class="fas fa-microphone-alt"></i>
                <h1>StellarSpeech</h1>
            </div>
            <p class="slogan">Bridging language barriers with real-time voice translation</p>
        </header>

        <div class="main-content">
            <div class="card">
                <h2><i class="fas fa-headset"></i> Audio Input</h2>

                <div class="form-group">
                    <label for="language">Select Language (or Auto-detect)</label>
                    <select id="language" name="language" class="language-select">
                        <option value="auto" selected>Auto-detect</option>
                        <option value="english">English</option>
                        <option value="hindi">Hindi</option>
                        <option value="telugu">Telugu</option>
                    </select>
                </div>

                <div class="form-group">
                    <div class="file-upload" id="dropArea">
                        <i class="fas fa-cloud-upload-alt"></i>
                        <p>Drop your audio file here or click to browse</p>
                        <input type="file" id="fileInput" class="file-upload-input" accept=".wav, .mp3">
                        <div id="fileInfo" class="file-info hidden"></div>
                    </div>
                </div>

                <div class="button-group">
                    <button id="uploadBtn" class="upload-btn" disabled>
                        <i class="fas fa-upload"></i> Upload
                    </button>
                    <button id="recordBtn" class="record-btn">
                        <i class="fas fa-microphone"></i> Record
                    </button>
                    <button id="stopBtn" class="stop-btn hidden">
                        <i class="fas fa-stop"></i> Stop
                    </button>
                </div>

                <div id="recordingIndicator" class="hidden">

                </div>
            </div>

            <div class="card">
                <h2><i class="fas fa-language"></i> Translation Results</h2>

                <div id="loader" class="loader hidden">
                    <div class="spinner"></div>
                </div>

                <div id="errorMessage" class="error-message hidden"></div>

                <div id="results" class="hidden">
                    <div id="emotionResult"></div>

                    <div class="result-card">
                        <h3><i class="fas fa-file-audio"></i> Original Speech</h3>
                        <div id="transcriptionResult" class="result-text"></div>
                        <div id="transcriptionAudio" class="audio-player"></div>
                    </div>

                    <div class="result-card">
                        <h3><i class="fas fa-exchange-alt"></i> Translation</h3>
                        <div id="translationResult" class="result-text"></div>
                        <div id="translationAudio" class="audio-player"></div>
                    </div>
                </div>
            </div>
        </div>

        <footer>
            <p>&copy; 2025 StellarSpeech. All rights reserved.</p>
        </footer>
    </div>

    <script>
// DOM elements
const fileInput = document.getElementById('fileInput');
const dropArea = document.getElementById('dropArea');
const fileInfo = document.getElementById('fileInfo');
const uploadBtn = document.getElementById('uploadBtn');
const recordBtn = document.getElementById('recordBtn');
const stopBtn = document.getElementById('stopBtn');
const recordingIndicator = document.getElementById('recordingIndicator');
const loader = document.getElementById('loader');
const errorMessage = document.getElementById('errorMessage');
const results = document.getElementById('results');
const emotionResult = document.getElementById('emotionResult');
const transcriptionResult = document.getElementById('transcriptionResult');
const translationResult = document.getElementById('translationResult');
const transcriptionAudio = document.getElementById('transcriptionAudio');
const translationAudio = document.getElementById('translationAudio');
const languageSelect = document.getElementById('language');
const logoutBtn = document.querySelector('.logout-btn');

// Logout button functionality
logoutBtn.addEventListener('click', function() {
    if(confirm('Are you sure you want to logout?')) {
        // Redirect to logout page or perform logout action
        alert('Logging out...');
        // window.location.href = '/logout';
    }
});

// Global variables
let mediaRecorder;
let audioChunks = [];
let audioBlob;
let isRecording = false;

// Ensure recording indicator is hidden at startup
recordingIndicator.classList.add('hidden');

// Event listeners for file drag and drop
['dragenter', 'dragover', 'dragleave', 'drop'].forEach(eventName => {
    dropArea.addEventListener(eventName, preventDefaults, false);
});

function preventDefaults(e) {
    e.preventDefault();
    e.stopPropagation();
}

['dragenter', 'dragover'].forEach(eventName => {
    dropArea.addEventListener(eventName, highlight, false);
});

['dragleave', 'drop'].forEach(eventName => {
    dropArea.addEventListener(eventName, unhighlight, false);
});

function highlight() {
    dropArea.style.borderColor = 'var(--primary-color)';
}

function unhighlight() {
    dropArea.style.borderColor = 'var(--border-color)';
}

// Handle file drop
dropArea.addEventListener('drop', handleDrop, false);

function handleDrop(e) {
    const dt = e.dataTransfer;
    const files = dt.files;

    if (files.length > 0) {
        handleFiles(files[0]);
    }
}

// Handle file selection
dropArea.addEventListener('click', () => {
    fileInput.click();
});

fileInput.addEventListener('change', (e) => {
    if (e.target.files.length > 0) {
        handleFiles(e.target.files[0]);
    }
});

function handleFiles(file) {
    // Check if file is an audio file
    const validTypes = ['audio/wav', 'audio/mp3', 'audio/mpeg'];
    if (file.type && !validTypes.includes(file.type)) {
        showError('Please select a valid audio file (WAV or MP3)');
        return;
    }

    // Display file info
    fileInfo.textContent = `${file.name || 'Audio file'} (${formatFileSize(file.size)})`;
    fileInfo.classList.remove('hidden');

    // Store the file
    audioBlob = file;

    // Enable upload button
    uploadBtn.disabled = false;
}

function formatFileSize(bytes) {
    if (bytes < 1024) {
        return bytes + ' bytes';
    } else if (bytes < 1024 * 1024) {
        return (bytes / 1024).toFixed(2) + ' KB';
    } else {
        return (bytes / (1024 * 1024)).toFixed(2) + ' MB';
    }
}

// Upload button click handler
uploadBtn.addEventListener('click', uploadAudio);

function uploadAudio() {
    if (!audioBlob) {
        showError('Please select an audio file first');
        return;
    }

    // Hide any previous results and show loader
    results.classList.add('hidden');
    errorMessage.classList.add('hidden');
    loader.classList.remove('hidden');

    // Create form data
    const formData = new FormData();

    // Fix: Add the file with a proper filename and MIME type
    const blobToUpload = new File([audioBlob], "recorded_audio.wav", {
        type: audioBlob.type || 'audio/wav'
    });

    formData.append('file', blobToUpload);
    formData.append('language', languageSelect.value);

    // Upload file
    fetch('/upload', {
        method: 'POST',
        body: formData
    })
    .then(response => {
        if (!response.ok) {
            throw new Error('Network response was not ok');
        }
        return response.json();
    })
    .then(data => {
        // Hide loader
        loader.classList.add('hidden');

        // Process results
        displayResults(data);
    })
    .catch(error => {
        loader.classList.add('hidden');
        showError('Error processing audio: ' + error.message);
        console.error('Upload error:', error);
    });
}

// Record audio functionality
recordBtn.addEventListener('click', startRecording);
stopBtn.addEventListener('click', stopRecording);

async function startRecording() {
    try {
        const stream = await navigator.mediaDevices.getUserMedia({ audio: true });

        mediaRecorder = new MediaRecorder(stream);
        audioChunks = [];

        mediaRecorder.addEventListener('dataavailable', event => {
            audioChunks.push(event.data);
        });

        mediaRecorder.addEventListener('stop', () => {
            // Fix: Properly create the audio blob with correct mimetype
            audioBlob = new Blob(audioChunks, { type: 'audio/wav' });

            // Display file info
            fileInfo.textContent = `Recorded audio (${formatFileSize(audioBlob.size)})`;
            fileInfo.classList.remove('hidden');

            // Enable upload button
            uploadBtn.disabled = false;

            // Reset recording UI
            recordBtn.classList.remove('hidden');
            stopBtn.classList.add('hidden');
            recordingIndicator.classList.add('hidden');
            isRecording = false;
        });

        mediaRecorder.start();
        isRecording = true;

        // Update UI
        recordBtn.classList.add('hidden');
        stopBtn.classList.remove('hidden');
        recordingIndicator.classList.remove('hidden');

    } catch (err) {
        showError('Error accessing microphone: ' + err.message);
    }
}

function stopRecording() {
    if (mediaRecorder && isRecording) {
        mediaRecorder.stop();
        // Fix: Ensure all tracks are properly stopped
        mediaRecorder.stream.getTracks().forEach(track => track.stop());
    }
}

function displayResults(data) {
    // Clear previous results
    emotionResult.innerHTML = '';
    transcriptionResult.innerHTML = '';
    translationResult.innerHTML = '';
    transcriptionAudio.innerHTML = '';
    translationAudio.innerHTML = '';

    // Display emotion
    const emotionIcon = getEmotionIcon(data.emotion);
    emotionResult.innerHTML = `
        <div class="emotion-tag">
            <i class="${emotionIcon}"></i> ${data.emotion}
        </div>
        <span class="language-tag">
            <i class="fas fa-globe"></i> ${capitalizeFirstLetter(data.detected_language)}
        </span>
    `;

    // Display transcription and translation
    transcriptionResult.textContent = data.transcription;
    translationResult.textContent = data.translation;

    // Add audio players if available
    if (data.transcription_audio) {
        const audioElement = document.createElement('audio');
        audioElement.controls = true;
        audioElement.src = data.transcription_audio;
        transcriptionAudio.appendChild(audioElement);
    }

    if (data.translation_audio) {
        const audioElement = document.createElement('audio');
        audioElement.controls = true;
        audioElement.src = data.translation_audio;
        translationAudio.appendChild(audioElement);
    }

    // Show results
    results.classList.remove('hidden');
}

function getEmotionIcon(emotion) {
    const emotionIcons = {
        'neutral': 'far fa-meh',
        'happy': 'far fa-smile',
        'sad': 'far fa-sad-tear',
        'angry': 'far fa-angry',
        'fear': 'far fa-grimace',
        'disgust': 'far fa-frown',
        'surprise': 'far fa-surprise'
    };

    return emotionIcons[emotion.toLowerCase()] || 'far fa-meh';
}

function showError(message) {
    errorMessage.textContent = message;
    errorMessage.classList.remove('hidden');
    loader.classList.add('hidden');
}

function capitalizeFirstLetter(string) {
    return string.charAt(0).toUpperCase() + string.slice(1);
}

// Make sure recording indicator is hidden on page load
document.addEventListener('DOMContentLoaded', function() {
    recordingIndicator.classList.add('hidden');
});
    </script>
</body>
</html>


   '''
    with open('templates/index.html', 'w') as f:
        f.write(html)
    return "Template generated successfully!"





if __name__ == "__main__":
    # Create templates directory if it doesn't exist
    os.makedirs('templates', exist_ok=True)
    init_db()
    generate_auth_templates()

    # Generate the HTML template
    generate_template()

    generate_settings_template()

    # Start ngrok tunnel
    public_url = ngrok.connect(5000)
    print(" * ngrok tunnel URL:", public_url)

    # Start Flask app
    app.run(port=5000)

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task 


--- Setting up Hindi-English Model ---

--- Setting up English-Hindi Model ---
Database initialized successfully!
 * ngrok tunnel URL: NgrokTunnel: "https://1ddf-34-85-221-25.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:19:19] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:19:19] "GET /signin HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:19:19] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:19:40] "POST /signin HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:19:53] "POST /signin HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:19:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:19:59] "GET /settings HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:20:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:20:17] "GET /logout HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:20:17] "GET /signin HTTP/1.1" 200 -
INFO:

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


...

INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:26:34] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:26:34] "GET /get_audio?file=transcription_english_1746131190.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:26:34] "GET /get_audio?file=translation_hi_1746131190.mp3 HTTP/1.1" 206 -
<ipython-input-21-0b08cb7b15be>:360: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:27:30] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:27:30] "GET /get_audio?file=translation_hi_1746131247.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:27:30] "GET /get_audio?file=transcription_english_1746131247.mp3 HTTP/1.1" 206 -
<ipython-input-21-0b08cb7b15be>:360: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:29:25] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:29:25] "GET /get_audio?file=transcription_english_1746131360.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:29:25] "GET /get_audio?file=translation_hi_1746131360.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:35:32] "GET /logout HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:35:33] "GET /signin HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/May/2025 20:35:36] "GET /signin HTTP/1.1" 200 -
